In [1]:
%load_ext autoreload
%autoreload 2

In [1]:

import os
import sys
#os.environ['HF_HOME'] = '/data/users/ugarg/hf/hf_cache/'
os.environ['TRANSFORMERS_CACHE'] = '/data/users/ugarg/hf/hf_cache/'
os.environ['CUDA_VISIBLE_DEVICES']='0'

#sys.path.append('../../..')
sys.path.append('../')
sys.path.append('./')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=False)
from torch.nn.functional import one_hot


from collections import Counter

from datasets import load_dataset, Dataset, DatasetDict
import pandas as pd
import torch
import numpy as np
import random
from transformers import AdamW, AutoTokenizer,  AutoModel
from torch.nn.functional import one_hot
from collections import Counter
from torch.optim import AdamW, Adam
from transformers import get_scheduler, get_cosine_with_hard_restarts_schedule_with_warmup

def seed_everything(seed: int):
    
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(42)
import joblib
import datetime
from tqdm.auto import tqdm


from utils.get_data_and_splits import LoadData
from utils.params import get_roberta_params, get_xlm_params
from utils.model import Model
from utils.utils import checkpoint_builder
from utils.faiss_utils import create_and_store_index, get_top_n_accuracy
from utils.predict import prep_model, predict_on_batch #

from CreatePytorchDataset import TrainDataset, ValDataset


INFO: Pandarallel will run on 48 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


2022-07-28 21:47:28.511683: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


Seed set to 42


In [2]:
loaddata = LoadData()
train, val, test = loaddata.get_data(
        extra_data=False,
        extra_data_path="/data/users/abose1/Capstone/Dictionary_and_Word_Embeddings/data/static_wiki.pkl"       
        ) #removes non-zero,

print (train[['word','gloss', 'fasttext']].info())

train.sample(6)

100%|██████████| 4529/4529 [00:00<00:00, 28849.97it/s]


Len of Train before drop duplicates - 28983
Len of Val before drop duplicates - 4219
Len of Test before drop duplicates - 4227
Len of Train - 28983
Len of Val - 4219
Len of Test - 4227
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28983 entries, 0 to 28982
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   word      28983 non-null  object
 1   gloss     28983 non-null  object
 2   fasttext  28983 non-null  object
dtypes: object(3)
memory usage: 679.4+ KB
None


,word,gloss,fasttext
27154,underground,A movement or organisation of people who resis...,"[-0.2013, -0.3014, 0.1525, 0.1562, -0.304, -0...."
20981,refection,"Physical refreshment , especially with food or...","[0.0731, 0.2905, -0.5642, -0.0788, -0.0255, 0...."
22957,shelter,To take cover .,"[-0.0753, -0.5014, -0.2117, -0.1908, -0.3724, ..."
13664,indecisive,inconclusive or uncertain,"[-0.0403, 0.0665, 0.1213, -0.7659, 0.5997, 0.0..."
2929,bedevil,To harass or cause trouble for ; to plague .,"[0.1295, -0.0428, 0.0991, -0.1387, 0.3032, 0.1..."
24343,stage,To place in position to prepare for use .,"[0.0975, -0.3233, -0.1921, -0.0952, 0.0622, 0...."


In [4]:
# train = joblib.load('../data/clean_data/train.joblib')
# val = joblib.load('../data/clean_data/val.joblib')
# test = joblib.load('../data/clean_data/test.joblib')

In [3]:
params = get_xlm_params()
params


{'model_checkpoint': 'xlm-roberta-large',
 'source_column': 'gloss',
 'max_len': 150,
 'batch_size': 64,
 'dropout': 0.1,
 'learning_rate': 0.001,
 'num_epochs': 150,
 'early_stopping_limit': 5,
 'device': 'cuda:0',
 'loss_fn_name': 'cosine',
 'emb_type': 'fasttext',
 'use_adapters': True,
 'resume_from_checkpoint': False,
 'output_size': 300,
 'knn_measure': 'cosine',
 'extra_data': False,
 'num_warmup_steps': 300}

In [4]:
# from utils.faiss_utils import create_and_store_index, get_top_n_accuracy

# create_and_store_index(
#     params['emb_type'], params['knn_measure'])


In [5]:

##################
#   tokenizer
##################

from transformers import AutoTokenizer

model_checkpoint = params['model_checkpoint']
print(f'\nLoading Tokenizer: {model_checkpoint} ...')
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)




Loading Tokenizer: xlm-roberta-large ...


In [15]:
save_checkpoint_path = f"../checkpoints/{params['model_checkpoint']}_{params['loss_fn_name']}loss_{params['emb_type']}_embs_{params['use_adapters']}_adapter_{params['extra_data']}_extradata"
save_checkpoint_path

i=1
while True:
    if os.path.exists(save_checkpoint_path):
        print (f'Path {save_checkpoint_path} already exists.')
        
        save_checkpoint_path = save_checkpoint_path + '_' + str(i)
        print(f'New path: {save_checkpoint_path}')
        i+=1
        
    else:
        os.mkdir(save_checkpoint_path)
        break



In [18]:
import json
json.dump(params,open(save_checkpoint_path+'/params.json','w'),indent=4)


In [6]:
train_dataset = TrainDataset(
    train, 
    tokenizer, 
    params['source_column'],
    params['max_len'], 
    params['emb_type']
)
val_dataset = ValDataset(
    val, 
    tokenizer, 
    params['source_column'],
    params['max_len'], 
    params['emb_type']
)

In [20]:
next(iter(train_dataloader))['attention_mask'].sum(axis=1)#.tolist()

tensor([23, 74, 33])

In [16]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

train_dataloader = DataLoader(
    train_dataset,
    shuffle=True,
    collate_fn = data_collator,
    batch_size = params['batch_size'],
)
val_dataloader = DataLoader(
    val_dataset, 
    collate_fn = data_collator, 
    batch_size=params['batch_size']
)
print (f'Train len {len(train_dataloader)}, val len: {len(val_dataloader)}')

Train len 9661, val len: 66


In [15]:

####################################
#   Load Model
####################################

print(f"\nUsing device {params['device']} for training...")

model = Model(params['model_checkpoint'], 
              params['output_size'], 
              params['dropout'], 
              params['device'], 
              params['loss_fn_name'], 
              params['use_adapters'])
model.to(params['device'])

1


Using device cuda:0 for training...


Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


1

In [16]:


####################################
#   Create Optimizer
####################################
print(f"\nCreating optimizer with Learning Rate: {params['learning_rate']}")


optimizer = AdamW(model.parameters(), lr = params['learning_rate'])

print(f"\nTraining for {params['num_epochs']} epochs with early stopping set to {params['early_stopping_limit']}\n\n")

num_train_epochs = params['num_epochs']
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_cosine_with_hard_restarts_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=300,
    num_training_steps=num_training_steps,
)



Creating optimizer with Learning Rate: 0.001

Training for 150 epochs with early stopping set to 5




In [17]:
params['resume_from_checkpoint']=False
if params['resume_from_checkpoint']:
    print('Resuming from Checkpoint...')
    print('Loading Checkpoint ...')
    checkpoint = torch.load(save_checkpoint_path, map_location=params['device'])
    print('Setting Models state to checkpoint...')
    #assert checkpoint['model_params'] == model_params
    model.load_state_dict(checkpoint['model_state_dict'])
    print('Model state set.')
    optimizer.load_state_dict(checkpoint['optim_state_dict'])
    print('Optimizer state set.')

In [18]:
progress_bar = tqdm(range(num_training_steps))
early_stopping_counter = 0
early_stopping_limit = params['early_stopping_limit']
best_valid_loss = float('inf')

for epoch in range(num_train_epochs):
    train_loss=0
    valid_loss =0
    

    
    print(f"[Epoch {epoch} / {num_train_epochs}]")
    # Training
    model.train()
    for batch_idx, batch in enumerate(train_dataloader):
        loss, out, actual = model(batch)
        #loss = outputs.loss
        loss.backward(loss)#, retain_graph = True)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        train_loss+= ((1 / (batch_idx + 1)) * (loss.data.item() - train_loss))
        progress_bar.update(1)
        progress_bar.set_postfix(loss = train_loss)
        
        
    
    
    # Evaluation
    model.eval()
    for batch_idx, batch in enumerate(val_dataloader):
        with torch.no_grad():
            loss, out, actual = model(batch)


        labels = actual
        predictions = out
        
        valid_loss+= ((1 / (batch_idx + 1)) * (loss.data.item() - valid_loss))
        

    print('\nEpoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))

    #early stopping, checkpointing
    if valid_loss < best_valid_loss:
        early_stopping_counter = 0
        best_valid_loss = valid_loss
        #create checkpoint
        print("Loss improved saving checkpoint... ")
        checkpoint_builder(model, optimizer, epoch, save_checkpoint_path)

    else:
        early_stopping_counter += 1
        if early_stopping_counter >= early_stopping_limit:
            print(f'\nLoss did not reduce for last {early_stopping_counter} epochs. Stopped training..')
            break



  0%|          | 0/67950 [00:00<?, ?it/s]

[Epoch 0 / 150]

Epoch: 0 	Training Loss: 0.623871 	Validation Loss: 0.524688
Loss improved saving checkpoint... 
[Epoch 1 / 150]

Epoch: 1 	Training Loss: 0.500304 	Validation Loss: 0.493887
Loss improved saving checkpoint... 
[Epoch 2 / 150]

Epoch: 2 	Training Loss: 0.475877 	Validation Loss: 0.485578
Loss improved saving checkpoint... 
[Epoch 3 / 150]

Epoch: 3 	Training Loss: 0.460090 	Validation Loss: 0.482507
Loss improved saving checkpoint... 
[Epoch 4 / 150]

Epoch: 4 	Training Loss: 0.447354 	Validation Loss: 0.480377
Loss improved saving checkpoint... 
[Epoch 5 / 150]

Epoch: 5 	Training Loss: 0.436885 	Validation Loss: 0.482273
[Epoch 6 / 150]

Epoch: 6 	Training Loss: 0.427460 	Validation Loss: 0.483772
[Epoch 7 / 150]

Epoch: 7 	Training Loss: 0.418930 	Validation Loss: 0.485243
[Epoch 8 / 150]

Epoch: 8 	Training Loss: 0.412001 	Validation Loss: 0.487656
[Epoch 9 / 150]

Epoch: 9 	Training Loss: 0.406038 	Validation Loss: 0.488560

Loss did not reduce for last 5 epochs. 